In [1]:
%run config.ipynb

In [2]:
from numpy import newaxis
import keras
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Convolution1D, MaxPooling1D, Flatten,  Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from datetime import datetime

Using TensorFlow backend.


In [3]:
# Connect to Cortex
cortex = Cortex.client()

In [4]:
# Our stock symbols
symbols = ['fb', 'xlf', 'aapl']

In [5]:
s = symbols[0]
print('Loading stock prices for %s' % s)
ds = cortex.dataset('demo/stock-prices-%s' % s)
c = ds.contract('stock-prices')
df = c.load_feature_file(tag='prices_%s' % s)
c.discover.data_dictionary(df)

Loading stock prices for fb


,Attribute,Type,% Nulls,Count,Unique,Observations,Knowledge
0,close,float64,0.0,1259,1200,max=2.4019017239971596 | min=-1.6881672929783968 | mean=-0.0,


In [6]:
look_back = 30
holdout_date = pd.to_datetime('2018-01-01')

train_df = df.loc[df.index < holdout_date]

# Convert our target variable to the right output format (2d array)
timeseries = np.asarray(train_df.close)
timeseries = np.atleast_2d(timeseries)
if timeseries.shape[0] == 1:
    timeseries = timeseries.T
    
timeseries.shape

(1114, 1)

In [7]:
# Setup training instances
X = np.atleast_3d(np.array([timeseries[start:start + look_back] for start in range(0, timeseries.shape[0] - look_back)]))
y = timeseries[look_back:]

print(X.shape)
print(X[0])
print(y.shape)
print(y[0:4])

(1084, 30, 1)
[[-1.6827422 ]
 [-1.66716766]
 [-1.6544865 ]
 [-1.62873991]
 [-1.64318422]
 [-1.63595076]
 [-1.64341026]
 [-1.64431445]
 [-1.65064372]
 [-1.6777918 ]
 [-1.68613288]
 [-1.68816729]
 [-1.67641292]
 [-1.65991159]
 [-1.64634886]
 [-1.64838327]
 [-1.64318422]
 [-1.5979751 ]
 [-1.5801175 ]
 [-1.61854525]
 [-1.59802031]
 [-1.58147377]
 [-1.58115731]
 [-1.56813708]
 [-1.57017149]
 [-1.55027948]
 [-1.5211196 ]
 [-1.51908519]
 [-1.52903119]
 [-1.49648063]]
(1084, 1)
[[-1.50303595]
 [-1.51298196]
 [-1.55367016]
 [-1.49580249]]


In [8]:
model = Sequential()
model.add(LSTM(input_shape = (1,), input_dim=1, output_dim=6, return_sequences=True))
model.add(LSTM(input_shape = (1,), input_dim=1, output_dim=6, return_sequences=False))
model.add(Dense(1))
model.add(Activation('linear'))
model.compile(loss="mse", optimizer="rmsprop")

In [ ]:
from keras_tqdm import TQDMNotebookCallback

callbacks = []
# callbacks.append(TQDMNotebookCallback(leave_inner=False, leave_outer=True))

model.fit(X, 
          y, 
          epochs=1000, 
          batch_size=80, shuffle=False, callbacks=callbacks)

In [11]:
# Save model
model.save(filepath='model_stock_prices_%s.h5' % s, overwrite=True)